
# Import Libraries


In [36]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns 
# this statement allows the visuals to render within your Jupyter Notebook
%matplotlib inline 
# You can configure the format of the images: ‘png’, ‘retina’, ‘jpeg’, ‘svg’, ‘pdf’.
%config InlineBackend.figure_format = 'png'

In [37]:
# Project for best sales in amazon.sa

In [38]:
# The best electroni sales (Page 1)

In [39]:
header = {'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
url = "https://www.amazon.sa/-/en/gp/bestsellers/electronics/?ie=UTF8&ref_=sv_sv_elec_all_1"

In [40]:
response = requests.get(url, {'headers':header})
response.status_code

200

In [41]:
response.text[:1000]

'<!doctype html><html lang="en-ae" class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n<!-- sp:end-feature:head-start -->\n\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'>\n<link rel="dns-prefetch" href="https://images-eu.ssl-images-amazon.com">\n<link rel="dns-prefetch" href="https://m.media-amazon.com">\n<link rel="dns-prefetch" href="https://completion.amazon.com">\n<!-- sp:end-feature:cs-optimization -->\n\n<!-- sp:feature:aui-assets -->\n<link rel="stylesheet" href="https://images-eu.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|01ZTHTZObnL.css,41JZEtDv4tL.css,31Y8m1dzTdL.css,013z33uKh2L.css,017DsKjNQJL.css,0131vqwP5UL.css,41EWOOlBJ9L.css,11TIuySqr6L.css,01ElnPiDxWL.css,11bGSgD5pDL.css,01Dm5eKVxwL.css,01IdKcBuAdL.css,01y-XAlI+2L.css,21N4kUH7pxL.css,01oDR3IULNL.css,41CYNGpGlrL.css,01XPHJk60-L.css,114y0SIP+yL.css,21aPhFy+riL.cs

# BeautifulSoup Basics

In [42]:
page = response.text
soup = BeautifulSoup(page, 'lxml')

In [43]:
print(soup.prettify())

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-ae">
 <!-- sp:feature:head-start -->
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <!-- sp:end-feature:head-start -->
  <!-- sp:feature:cs-optimization -->
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetch"/>
  <link href="https://m.media-amazon.com" rel="dns-prefetch"/>
  <link href="https://completion.amazon.com" rel="dns-prefetch"/>
  <!-- sp:end-feature:cs-optimization -->
  <!-- sp:feature:aui-assets -->
  <link href="https://images-eu.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|01ZTHTZObnL.css,41JZEtDv4tL.css,31Y8m1dzTdL.css,013z33uKh2L.css,017DsKjNQJL.css,0131vqwP5UL.css,41EWOOlBJ9L.css,11TIuySqr6L.css,01ElnPiDxWL.css,11bGSgD5pDL.css,01Dm5eKVxwL.css,01IdKcBuAdL.css,01y-XAlI+2L.css,21N4kUH7pxL.css,01oDR3IULNL.css,41CYNGpGlrL.css,01XPHJk60-L.css,114y0SIP+yL.css,21aPhFy+riL

In [44]:
soup.find_all('li', {'class':'zg-item-immersion'})[0].find_all('a')[1].text

'\n3.0 out of 5 stars\n'

In [45]:
# create function to clean data 

In [46]:
def clean_num(s):
    """cleans text scraped from website
    
    Parameters:
    s (String): raw text

    Returns:
    String: cleaned digit text

    """ 

    if s is None:
        return s
    return "".join(re.findall('\d*\.?\d+',s))

def get_data(block, tag, class_text):
    """fetch data from tag and its associated class
    
    Parameters:
    block (int): Book item 
    tag (String): tag for element we want to fetch
    class_text (String): class for element we want to fetch

    Returns:
    String: data text fetched

    """ 
    try:
        return block.find(tag, class_ = class_text).text
    except AttributeError:
        return ""
    return

In [47]:
#Test with the first electroni product to ensure functions are working
for i in (soup.find("li", "zg-item-immersion")):
    print(str.strip(get_data(i, "a", "a-link-normal")))
    print(get_data(i, "span", "a-icon-alt"))
    print(clean_num(get_data(i, "span", "a-icon-alt").split(" ")[0]))
    print(get_data(i, "a","a-size-small a-link-normal"))
    print(clean_num(get_data(i, "a","a-size-small a-link-normal")))
    print(get_data(i, "span","p13n-sc-price"))
    print(clean_num(get_data(i, "span", "p13n-sc-price")))


SAMSUNG Galaxy A12 Dual SIM Smartphone - 64GB, 4GB RAM, LTE, Black (KSA Version)
3.0 out of 5 stars
3.0
2
2
SAR 539.00
539.00


In [48]:
soup.find_all("li", class_ = "zg-item-immersion")[3] 

<li class="zg-item-immersion" role="gridcell"><span class="a-list-item"><div class="a-section a-spacing-none aok-relative"><div class="a-row a-spacing-none aok-inline-block"><span class="a-size-small aok-float-left zg-badge-body zg-badge-color"><span class="zg-badge-text">#4</span></span><span class="aok-float-left zg-badge-triangle zg-badge-color"></span></div><span class="aok-inline-block zg-item"><a class="a-link-normal" href="/-/en/Apple-USB-C-Power-Adapter-White/dp/B08L85KDV5?_encoding=UTF8&amp;psc=1"><span class="zg-text-center-align"><div class="a-section a-spacing-small"><img alt="Apple 20W USB-C Power Adapter, White" height="200" src="https://images-na.ssl-images-amazon.com/images/I/61TbaDn9AgS._AC_UL200_SR200,200_.jpg" width="200"/></div></span>
<div aria-hidden="true" class="p13n-sc-truncate p13n-sc-line-clamp-2" data-rows="2">
            Apple 20W USB-C Power Adapter, White
        </div>
</a>
<div class="a-icon-row a-spacing-none">
<a class="a-link-normal" href="/-/en/pro

In [49]:
products_list = []
for i in (soup.find_all("li", class_ = "zg-item-immersion")):
    pro_name = (str.strip(get_data(i, "a", "a-link-normal")))
    pro_rate= (get_data(i, "span", "a-icon-alt"))
    pro_rate_clean = (clean_num(get_data(i, "span", "a-icon-alt").split(" ")[0]))
    review_num = (get_data(i, "a","a-size-small a-link-normal"))
    review_num_clean = (clean_num(get_data(i, "a","a-size-small a-link-normal")))
    pro_price = (get_data(i, "span","p13n-sc-price"))
    pro_price_clean = (clean_num(get_data(i, "span", "p13n-sc-price")))
    
    headers = ['pro_name', 'pro_rate', 'pro_rate_clean','review_num',
              'review_num_clean', 'pro_price','pro_price_clean']
    products_dict = dict(zip(headers, [ pro_name,
                                    pro_rate,
                                    pro_rate_clean,
                                    review_num,
                                    review_num_clean, 
                                    pro_price,
                                    pro_price_clean]))
    products_list.append(products_dict)

    

In [50]:
products_list

[{'pro_name': 'SAMSUNG Galaxy A12 Dual SIM Smartphone - 64GB, 4GB RAM, LTE, Black (KSA Version)',
  'pro_rate': '3.0 out of 5 stars',
  'pro_rate_clean': '3.0',
  'review_num': '2',
  'review_num_clean': '2',
  'pro_price': 'SAR 539.00',
  'pro_price_clean': '539.00'},
 {'pro_name': 'Samsung Galaxy A12 LTE Dual SIM Smartphone - 64GB Storage, 4GB RAM, Blue (KSA Version)',
  'pro_rate': '1.0 out of 5 stars',
  'pro_rate_clean': '1.0',
  'review_num': '2',
  'review_num_clean': '2',
  'pro_price': 'SAR 539.00',
  'pro_price_clean': '539.00'},
 {'pro_name': 'SAMSUNG Galaxy M12 Dual SIM Smartphone - 64GB, 4GB RAM, 4G LTE, Black (KSA Version)',
  'pro_rate': '4.0 out of 5 stars',
  'pro_rate_clean': '4.0',
  'review_num': '38',
  'review_num_clean': '38',
  'pro_price': 'SAR 499.00',
  'pro_price_clean': '499.00'},
 {'pro_name': 'Apple 20W USB-C Power Adapter, White',
  'pro_rate': '4.5 out of 5 stars',
  'pro_rate_clean': '4.5',
  'review_num': '358',
  'review_num_clean': '358',
  'pro_pri

In [51]:
products_df = pd.DataFrame(products_list).replace("",np.nan)  #convert list of dict to df
products_df

,pro_name,pro_rate,pro_rate_clean,review_num,review_num_clean,pro_price,pro_price_clean
0,"SAMSUNG Galaxy A12 Dual SIM Smartphone - 64GB,...",3.0 out of 5 stars,3.0,2,2,SAR 539.00,539.00
1,Samsung Galaxy A12 LTE Dual SIM Smartphone - 6...,1.0 out of 5 stars,1.0,2,2,SAR 539.00,539.00
2,"SAMSUNG Galaxy M12 Dual SIM Smartphone - 64GB,...",4.0 out of 5 stars,4.0,38,38,SAR 499.00,499.00
3,"Apple 20W USB-C Power Adapter, White",4.5 out of 5 stars,4.5,358,358,SAR 79.00,79.00
4,"SAMSUNG Galaxy M12 Dual SIM Smartphone - 64GB,...",4.2 out of 5 stars,4.2,22,22,SAR 499.00,499.00
5,"SAMSUNG Galaxy M12 Dual SIM Smartphone - 64GB,...",4.5 out of 5 stars,4.5,10,10,SAR 499.00,499.00
6,New Apple Airpods Pro,4.3 out of 5 stars,4.3,"2,348",2348,SAR 749.00,749.00
7,"Anker PowerCore Select 20000, 20000mAh Power B...",4.1 out of 5 stars,4.1,54,54,SAR 94.00,94.00
8,Apple AirPods with Charging Case,4.1 out of 5 stars,4.1,"2,773",2773,SAR 475.00,475.00
9,Apple iPhone 12 With FaceTime (128GB) - White,4.4 out of 5 stars,4.4,353,353,"SAR 3,199.00",3199.00


In [52]:
# The best electroni sales (Page 2)

In [53]:
url = 'https://www.amazon.sa/-/en/gp/bestsellers/electronics/ref=zg_bs_pg_2?ie=UTF8&pg=2'

In [54]:
response = requests.get(url)
response.status_code

200

In [55]:
response.text[:1000]

'<!doctype html><html lang="en-ae" class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n<!-- sp:end-feature:head-start -->\n\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'>\n<link rel="dns-prefetch" href="https://images-eu.ssl-images-amazon.com">\n<link rel="dns-prefetch" href="https://m.media-amazon.com">\n<link rel="dns-prefetch" href="https://completion.amazon.com">\n<!-- sp:end-feature:cs-optimization -->\n\n<!-- sp:feature:aui-assets -->\n<link rel="stylesheet" href="https://images-eu.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|01ZTHTZObnL.css,41JZEtDv4tL.css,31Y8m1dzTdL.css,013z33uKh2L.css,017DsKjNQJL.css,0131vqwP5UL.css,41EWOOlBJ9L.css,11TIuySqr6L.css,01ElnPiDxWL.css,11bGSgD5pDL.css,01Dm5eKVxwL.css,01IdKcBuAdL.css,01y-XAlI+2L.css,21N4kUH7pxL.css,01oDR3IULNL.css,41CYNGpGlrL.css,01XPHJk60-L.css,114y0SIP+yL.css,21aPhFy+riL.cs

In [56]:
page = response.text
soup = BeautifulSoup(page, 'lxml')

In [57]:
print(soup.prettify())

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-ae">
 <!-- sp:feature:head-start -->
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <!-- sp:end-feature:head-start -->
  <!-- sp:feature:cs-optimization -->
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetch"/>
  <link href="https://m.media-amazon.com" rel="dns-prefetch"/>
  <link href="https://completion.amazon.com" rel="dns-prefetch"/>
  <!-- sp:end-feature:cs-optimization -->
  <!-- sp:feature:aui-assets -->
  <link href="https://images-eu.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|01ZTHTZObnL.css,41JZEtDv4tL.css,31Y8m1dzTdL.css,013z33uKh2L.css,017DsKjNQJL.css,0131vqwP5UL.css,41EWOOlBJ9L.css,11TIuySqr6L.css,01ElnPiDxWL.css,11bGSgD5pDL.css,01Dm5eKVxwL.css,01IdKcBuAdL.css,01y-XAlI+2L.css,21N4kUH7pxL.css,01oDR3IULNL.css,41CYNGpGlrL.css,01XPHJk60-L.css,114y0SIP+yL.css,21aPhFy+riL

In [58]:
soup.find_all('li', {'class':'zg-item-immersion'})[1].find_all('a')[1].text

'\n4.2 out of 5 stars\n'

In [59]:
def clean_num(s):
    """cleans text scraped from website
    
    Parameters:
    s (String): raw text

    Returns:
    String: cleaned digit text

    """ 

    if s is None:
        return s
    return "".join(re.findall('\d*\.?\d+',s))

def get_data(block, tag, class_text):
    """fetch data from tag and its associated class
    
    Parameters:
    block (int): Book item 
    tag (String): tag for element we want to fetch
    class_text (String): class for element we want to fetch

    Returns:
    String: data text fetched

    """ 
    try:
        return block.find(tag, class_ = class_text).text
    except AttributeError:
        return ""
    return

In [60]:
for i in (soup.find("li", "zg-item-immersion")):
    print(str.strip(get_data(i, "a", "a-link-normal")))
    print(get_data(i, "span", "a-icon-alt"))
    print(clean_num(get_data(i, "span", "a-icon-alt").split(" ")[0]))
    print(get_data(i, "a","a-size-small a-link-normal"))
    print(clean_num(get_data(i, "a","a-size-small a-link-normal")))
    print(get_data(i, "span","p13n-sc-price"))
    print(clean_num(get_data(i, "span", "p13n-sc-price")))

TP-Link AC1750 Wi-Fi Range Extender - RE450, White
4.0 out of 5 stars
4.0
5,915
5915
SAR 190.00
190.00


In [61]:
soup.find_all("li", class_ = "zg-item-immersion")[1] 

<li class="zg-item-immersion" role="gridcell"><span class="a-list-item"><div class="a-section a-spacing-none aok-relative"><div class="a-row a-spacing-none aok-inline-block"><span class="a-size-small aok-float-left zg-badge-body zg-badge-color"><span class="zg-badge-text">#52</span></span><span class="aok-float-left zg-badge-triangle zg-badge-color"></span></div><span class="aok-inline-block zg-item"><a class="a-link-normal" href="/-/en/SAMSUNG-Wireless-Earbuds-Intelligent-Canceling/dp/B08TCD9BBS?_encoding=UTF8&amp;psc=1"><span class="zg-text-center-align"><div class="a-section a-spacing-small"><img alt="SAMSUNG Galaxy Buds Pro, True Wireless Earbuds w/Intelligent Active Noise Canceling, Phantom Black" height="200" src="https://images-na.ssl-images-amazon.com/images/I/719p0eyZiSL._AC_UL200_SR200,200_.jpg" width="200"/></div></span>
<div aria-hidden="true" class="p13n-sc-truncate p13n-sc-line-clamp-2" data-rows="2">
            SAMSUNG Galaxy Buds Pro, True Wireless Earbuds w/Intelligen

In [62]:
products_list = []
for i in (soup.find_all("li", class_ = "zg-item-immersion")):
    pro_name = (str.strip(get_data(i, "a", "a-link-normal")))
    pro_rate= (get_data(i, "span", "a-icon-alt"))
    pro_rate_clean = (clean_num(get_data(i, "span", "a-icon-alt").split(" ")[0]))
    review_num = (get_data(i, "a","a-size-small a-link-normal"))
    review_num_clean = (clean_num(get_data(i, "a","a-size-small a-link-normal")))
    pro_price = (get_data(i, "span","p13n-sc-price"))
    pro_price_clean = (clean_num(get_data(i, "span", "p13n-sc-price")))
    
    headers = ['pro_name', 'pro_rate', 'pro_rate_clean','review_num',
              'review_num_clean', 'pro_price','pro_price_clean']
    products_dict = dict(zip(headers, [ pro_name,
                                    pro_rate,
                                    pro_rate_clean,
                                    review_num,
                                    review_num_clean, 
                                    pro_price,
                                    pro_price_clean]))
    products_list.append(products_dict)

In [63]:
products_df = pd.DataFrame(products_list).replace("",np.nan)  #convert list of dict to df
products_df

,pro_name,pro_rate,pro_rate_clean,review_num,review_num_clean,pro_price,pro_price_clean
0,"TP-Link AC1750 Wi-Fi Range Extender - RE450, W...",4.0 out of 5 stars,4.0,"5,915",5915,SAR 190.00,190.00
1,"SAMSUNG Galaxy Buds Pro, True Wireless Earbuds...",4.2 out of 5 stars,4.2,119,119,SAR 580.00,580.00
2,Glorious Large Gaming Mouse Pad 11''x13'' - Black,4.7 out of 5 stars,4.7,"7,988",7988,SAR 37.00,37.00
3,Extension Cord with 3 Power Socket and 3 USB S...,3.9 out of 5 stars,3.9,41,41,SAR 59.99,59.99
4,Long ARM Black Color Flexible Clip Lazy Bed De...,3.4 out of 5 stars,3.4,180,180,SAR 10.18,10.18
5,"Apple Watch Series 6 (GPS, 44mm) - Space Grey ...",4.6 out of 5 stars,4.6,306,306,"SAR 1,449.00",1449.00
6,Sony MDR-ZX110AP Extra Bass Smartphone Headset...,4.1 out of 5 stars,4.1,"3,890",3890,SAR 47.00,47.00
7,New Apple AirTag,4.7 out of 5 stars,4.7,23,23,SAR 89.00,89.00
8,"Xiaomi Redmi Note 10S, Dual SIM, 128GB, 6GB RA...",3.8 out of 5 stars,3.8,13,13,SAR 778.00,778.00
9,Samsung EP-TA800 Travel Adapter for Super Fast...,4.4 out of 5 stars,4.4,139,139,SAR 79.00,79.00
